In [5]:
import os
import sys

notebook_dir = os.getcwd() 

# Navigate up to the 'src' directory, then add 'project_ideate'
# Assuming your notebook is in 'notebook/' and 'src' is a sibling of 'notebook'
project_root = os.path.abspath(os.path.join(notebook_dir, '..')) # Goes from 'notebook' to project root
sys.path.append(os.path.join(project_root, 'src'))

# Agent Testing Notebook

This notebook provides a basic structure for testing the agents defined in:
* `src/project_ideate/agents/learning_agents.py`
* `src/project_ideate/agents/building_agents.py`
* `src/project_ideate/agents/organizing_agents.py`


## Import necessary libraries and agents

In [9]:
# Import agents from learning_agents.py
from project_ideate.agents.learning_agents import anthropologist, cross_pollinator, experimenter

# Import any other necessary libraries (e.g., for running agents, handling outputs)

ValidationError: 1 validation error for LlmAgent
name
  Value error, Found invalid agent name: `Cross-Pollinator`. Agent name must be a valid identifier. It should start with a letter (a-z, A-Z) or an underscore (_), and can only contain letters, digits (0-9), and underscores. [type=value_error, input_value='Cross-Pollinator', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/value_error

## Define a test function or class